In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import log10
%matplotlib inline

In [3]:
from ipywidgets import interact,interactive,interact_manual
from skimage import data
from skimage.color import rgb2gray
from numpy.linalg import svd

In [4]:
from skimage import img_as_ubyte,img_as_float
gray_images = {
        "cat":rgb2gray(img_as_float(data.chelsea())),
        "astro":rgb2gray(img_as_float(data.astronaut())),
}

In [5]:
def compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    plt.imshow(image,cmap='gray')
    U,s,V = svd(image,full_matrices=False)
    #U[:,:k] all rows but first k columns
    #s[:k] all k*k matrices
    #V[:k,:] all coulmns for first k rows
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
    return reconst_matrix,s

In [6]:
def mean_squared_error(im1, im2):
    sq_diff = (im1 - im2) ** 2
    s = np.sum(sq_diff)
    mse = s / (np.size(im1,0) * np.size(im1,1))
    return mse

In [7]:
def compress_show_gray_images(img_name,k):
    """
     compresses gray scale images and display the reconstructed image.
     Also displays a plot of singular values
    """
    image=gray_images[img_name]
    original_shape = image.shape
    print(original_shape)
    reconst_img,s = compress_svd(image,k)
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    axes[0].plot(s)
    #print(s)
    ko=(original_shape[0]*original_shape[1])/(k*(original_shape[0] + original_shape[1])+k)
    compression_ratio =ko
    space_saving=1-1/compression_ratio
    mse=mean_squared_error(image,reconst_img)
    psnr=10*log10(255*255/mse)
    s="compression ratio= "+str(compression_ratio)+"\nspace_saving= "+str(space_saving)+"\nmse"+str(mse)+"\npsnr"+str(psnr)
    axes[1].set_title(s)
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('off')
    fig.tight_layout()

In [8]:
interact(compress_show_gray_images,img_name=list(gray_images.keys()),k=(1,300));

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro'), value='cat'), IntSlider(value…

In [12]:
color_images={
    "cat":img_as_float(data.chelsea()),
    "astro":img_as_float(data.astronaut()),
    "coffee":img_as_float(data.coffee())
}

In [17]:
def compress_col_images(img_name,k):
    image=color_images[img_name]
    original_shape=image.shape
    print(original_shape)
    image_reshaped=image.reshape((original_shape[0],original_shape[1]*3))
    image_reconst,s=compress_svd(image_reshaped,k)
    image_reconst=image_reconst.reshape(original_shape)
    ko=(original_shape[0]*original_shape[1])/(k*(original_shape[0] + original_shape[1]+1))
    compression_ratio =ko
    mse=mean_squared_error(image,image_reconst)
    value=log10(16777215*16777215)
    psnr=10*(value-log10(mse))
    space_saving=1-1/compression_ratio
    print(compression_ratio,space_saving,mse,psnr)
    plt.title("space_saving "+str(space_saving))
    plt.imshow(image_reconst)

In [18]:
interact(compress_col_images,img_name=list(color_images.keys()),k=(1,300))

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro', 'coffee'), value='cat'), IntSl…

<function __main__.compress_col_images(img_name, k)>